<a href="https://colab.research.google.com/github/shubhangij12/shubhangi.github.io/blob/master/Language_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk 
import nltk.corpus
import os
import string
from string import digits
import matplotlib.pyplot as plt
import re
from keras.layers import Input, LSTM, Embedding, Dense, Layer, TimeDistributed
from keras.models import Model
from nltk.tokenize import sent_tokenize
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.activations import sigmoid
import tensorflow as tf
from nltk.util import ngrams
from keras.layers import Activation, dot, concatenate

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
EngHin=pd.read_excel(r"Hi-En-Parallel_Corpus.xlsx",encoding='utf-8')

In [ ]:
EngHin.dropna(inplace = True)

In [ ]:
EngHin['english_sentence'] = EngHin['english_sentence'].apply(str)
EngHin['hindi_sentence'] = EngHin['hindi_sentence'].apply(str)

In [ ]:
EngHin['english_sentence'] = EngHin['english_sentence'].apply(lambda x: x.lower())
EngHin['hindi_sentence'] = EngHin['hindi_sentence'].apply(lambda x: x.lower())

In [ ]:
exclude = set(string.punctuation)
EngHin['english_sentence']=EngHin['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
EngHin['hindi_sentence']=EngHin['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
dig = str.maketrans('', '', digits)
EngHin['english_sentence']=EngHin['english_sentence'].apply(lambda x: x.translate(dig))
EngHin['hindi_sentence']=EngHin['hindi_sentence'].apply(lambda x: x.translate(dig))

EngHin['hindi_sentence'] = EngHin['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))


In [ ]:
EngHin['english_sentence']=EngHin['english_sentence'].apply(lambda x: x.strip())
EngHin['hindi_sentence']=EngHin['hindi_sentence'].apply(lambda x: x.strip())
EngHin['english_sentence']=EngHin['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
EngHin['hindi_sentence']=EngHin['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [ ]:
EngHin['hindi_sentence'] = EngHin['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
all_eng_words=set()
for eng in EngHin['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in EngHin['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

Encoder - Decoder Model


**Architecture:**

*   Encoder-decoder model helps in finding the prediction of sequence-to-sequence problems in machine translation.
*   The most basic understanding of the architecture is that there are 2 parts of the model: One is the encoder and the other id the decoder.
*   In the encoder, we encode the input sequence given to us and in the decoder , we decode the encoded sequence which is given to us.
*   In this problem statement, we took the input english statements and preprocessed the data and finally got statements which only contained strings.We did the same with out target data and produced hindi strings. Now we collected all the english and hindi words in a list and named in all_english_words and all_hindi_words respectively.
*   We then divided the data into train set and test set. We saved these sets and then generated a batch of data.
*   Then we created a generic encoder and decoder models. In the encoder model, we took input from the dataset and we feeded the encoded output as an input in the decoder model and finally, defined the model.
*   We compile the model and as we have to classify the instances into 2 classes, we use the multiclass classifier thus we apply rmsprop optimiser and categorical_crossentropy loss.
*   We defined the number of epochs as 100 so that the learning algorithm can run on the dataset a 100 times to train the data.
*   We used a fit_generator for the model to be trained batch by batch.
*   After learning the weight values through training, we save the weights using the save_weights() function and the file is saved in the h5 format thus saving the data in a hierarchical form.
*   We finally make a decode_sequence function which can be used to decode the input sequence which is passed in the function.
*   We then make predictions on the datasets.













In [ ]:
EngHin['length_eng_sentence']=EngHin['english_sentence'].apply(lambda x:len(x.split(" ")))
EngHin['length_hin_sentence']=EngHin['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [ ]:
EngHin=EngHin[EngHin['length_eng_sentence']<=10]
EngHin=EngHin[EngHin['length_hin_sentence']<=10]

In [ ]:
print("maximum length of Hindi Sentence ",max(EngHin['length_hin_sentence']))
print("maximum length of English Sentence ",max(EngHin['length_eng_sentence']))

maximum length of Hindi Sentence  10
maximum length of English Sentence  10


In [ ]:
max_length_src=max(EngHin['length_hin_sentence'])
max_length_tar=max(EngHin['length_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(72686, 81977)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
EngHin = shuffle(EngHin)


In [ ]:
X, y = EngHin['english_sentence'], EngHin['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((30565,), (7642,))

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [ ]:
del(EngHin['hindi_sentence'])
del(EngHin['english_sentence'])

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim=300

In [ ]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    21806100    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    24593400    input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
  1/238 [..............................] - ETA: 51:20 - loss: 7.0006

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.load_weights('nmt_weights.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
scores=[]
opt=[]

BLEU SCORE

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
for k in range(0,len(X_test)):
  (input_seq, actual_output), _ = next(test_gen)
  decoded_sentence = decode_sequence(input_seq)
  reference = [y_test[k:k+1].values[0][6:-4].split(),y_test[k:k+1].values[0][6:-4].split()]
  candidate = decoded_sentence[:-4].split()
  score = sentence_bleu(reference, candidate)
  scores.append(score)
  opt.append(decoded_sentence[:-4])

In [ ]:
np.mean(scores)

Sample input statement 

In [ ]:
input_seq = "and then we look at each other"

In [ ]:
encoder_input_data = np.zeros((1, max_length_src),dtype='float32')

In [ ]:
for t, word in enumerate(input_seq.split()):
  encoder_input_data[0, t] = input_token_index[word]

In [ ]:
decoded_sentence = decode_sequence(encoder_input_data)

In [ ]:
decoded_sentence[:-4]